In [42]:
import sqlite3
import numpy as np
import pandas as pd
from sqlite3 import Error
import math
import blpapi
import pdblp
from datetime import date
from Database import PySQLite

DB = PySQLite("test1228v1.db")
currentYear = 2020.25
currentQuarter = 2

pd.options.display.float_format = '{:,.2f}'.format
float_formatter = "{:.2f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})






class CorporateCredit():    
    def __init__(self,name):     
        self.name = name
    
    def test(self):
        print('test')
        
    def makeDateColumn(self,qtrs):
        yearColumns = ['Q'+str(currentQuarter)+'/'+str(math.floor(currentYear))]
        for x in range (qtrs):
            yearColumns.append('Q'+str(((currentQuarter+x) % 4)+1)+'/'+str(math.floor(currentYear+(x+1)*0.25)))
        
        return yearColumns
    
    def FCF_Proj(self,qtrs):
        FCF = ['Revenue','Variable Costs','Fixed Costs','EBITDA', 'Less: Capex','Less: Interest', 'Less: Working Cap', 'Less: Taxes','Less: Pfd Div','Less: Common Div', 'FCF', 'Beginning Cash', 'Ending Cash'] 

        revArray = np.array([self.CurrentFCF['rev']*(1+self.CurrentFCF['rev_g'])**i for i in range(qtrs+1)],dtype=int)
        vcostsArray = np.array([self.CurrentFCF['var_costs']*(1+self.CurrentFCF['rev_g'])**i for i in range(qtrs+1)],dtype=int)
        fcostsArray = np.full(shape=qtrs+1,fill_value=self.CurrentFCF['f_costs'],dtype=int)
        ebitdaArray = revArray-vcostsArray-fcostsArray
        capexArray = revArray * self.CurrentFCF['capex/rev']
        
        interestArray = np.full(shape=qtrs+1,fill_value=self.CurrentFCF['interest'],dtype=int)               
        pfdArray = np.full(shape=qtrs+1,fill_value=self.CurrentFCF['pfd_distro'],dtype=int)                
        CommonArray = np.full(shape=qtrs+1,fill_value=self.CurrentFCF['common_distro'],dtype=int)                
        
        wcArray = revArray * self.CurrentFCF['WC/rev']
        taxArray = (ebitdaArray - interestArray)*self.CurrentFCF['tax']  #to fix --> need D&A
        fcfArray = ebitdaArray - capexArray - wcArray - taxArray - interestArray - pfdArray - CommonArray # need interest expense

        begCashArray = np.zeros(qtrs+1,dtype=int)
        begCashArray[0] = self.CurrentFCF['cash']
        endCashArray = np.zeros(qtrs+1,dtype=int)
        endCashArray[0] = fcfArray[0]+begCashArray[0]


        for i in range(1,qtrs+1):
            begCashArray[i] = endCashArray[i-1]
            endCashArray[i] = begCashArray[i] + fcfArray[i]


        FCF_statement=[revArray,vcostsArray,fcostsArray,ebitdaArray,capexArray,interestArray,wcArray,taxArray,pfdArray,CommonArray,fcfArray,begCashArray,endCashArray]
        FCF_df=pd.DataFrame(FCF_statement,columns = self.makeDateColumn(qtrs), index = FCF, dtype=int)


        print(fcfArray)
        return FCF_df
        

def main():
    ETP=CorporateCredit('ETP')
    ETP.CurrentFCF = {'rev': 100, 'rev_g': 0.05, 'var_costs': 2.7, 'f_costs': 3.9, 'cash': 4, 'capex/rev': 5,'interest':6,'pfd_distro':7,'common_distro':8,'WC/rev': 9, 'tax':1000}
    x=ETP.FCF_Proj(5)
    print(type(x))
    print(x)
    return(x)
    

    
a=main()
a
     

[ -90326  -95391 -100456 -104522 -110600 -116678]
<class 'pandas.core.frame.DataFrame'>
                   Q2/2020  Q3/2020  Q4/2020  Q1/2021  Q2/2021  Q3/2021
Revenue                100      105      110      115      121      127
Variable Costs           2        2        2        3        3        3
Fixed Costs              3        3        3        3        3        3
EBITDA                  95      100      105      109      115      121
Less: Capex            500      525      550      575      605      635
Less: Interest           6        6        6        6        6        6
Less: Working Cap      900      945      990     1035     1089     1143
Less: Taxes          89000    94000    99000   103000   109000   115000
Less: Pfd Div            7        7        7        7        7        7
Less: Common Div         8        8        8        8        8        8
FCF                 -90326   -95391  -100456  -104522  -110600  -116678
Beginning Cash           4   -90322  -185713  -2

,Q2/2020,Q3/2020,Q4/2020,Q1/2021,Q2/2021,Q3/2021
Revenue,100,105,110,115,121,127
Variable Costs,2,2,2,3,3,3
Fixed Costs,3,3,3,3,3,3
EBITDA,95,100,105,109,115,121
Less: Capex,500,525,550,575,605,635
Less: Interest,6,6,6,6,6,6
Less: Working Cap,900,945,990,1035,1089,1143
Less: Taxes,89000,94000,99000,103000,109000,115000
Less: Pfd Div,7,7,7,7,7,7
Less: Common Div,8,8,8,8,8,8
